In [ ]:
import os
import sys
import argparse
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm
from osgeo import gdal
from os.path import dirname as up

Tutorial : https://www.gis.usu.edu/~chrisg/python/2009/lectures/ospy_slides4.pdf \
Affine transformation "GetGeoTransform" : https://gdal.org/user/raster_data_model.html \
Intersection function : https://sciience.tumblr.com/post/101722591382/finding-the-georeferenced-intersection-between-two

In [ ]:
RefImage = '/data/sushen/marinedebris/MARIDA/patches/S2_1-12-19_48MYU/S2_1-12-19_48MYU_0.tif'
ds = gdal.Open(RefImage)
IM = np.copy(ds.ReadAsArray())
print(IM.shape)

# Read associated confidence level patch
ds_conf = gdal.Open(os.path.join(up(RefImage), '_'.join(os.path.basename(RefImage).split('.tif')[0].split('_')[:4]) + '_conf.tif'))
IM_conf = np.copy(ds_conf.ReadAsArray())[np.newaxis, :, :]
print(IM_conf.shape)

# Read associated class patch
ds_cl = gdal.Open(os.path.join(up(RefImage), '_'.join(os.path.basename(RefImage).split('.tif')[0].split('_')[:4]) + '_cl.tif'))
IM_cl = np.copy(ds_cl.ReadAsArray())[np.newaxis, :, :]
print(IM_cl.shape)

IM_T = np.moveaxis(np.concatenate([IM, IM_conf, IM_cl], axis = 0), 0, -1)
print(IM_T.shape)

In [ ]:
padfTransform = ds.GetGeoTransform()
print('Transform is: ', padfTransform)

y_coords, x_coords = np.meshgrid(range(IM_T.shape[0]), range(IM_T.shape[1]), indexing='ij')

Xp = padfTransform[0] + x_coords*padfTransform[1] + y_coords*padfTransform[2]
Yp = padfTransform[3] + x_coords*padfTransform[4] + y_coords*padfTransform[5]

# shift to the center of the pixel
Xp -= padfTransform[5] / 2.0
Yp -= padfTransform[1] / 2.0
print('Xp shape is: ', Xp.shape)
print('Yp shape is: ', Yp.shape)

XpYp = np.dstack((Xp,Yp))
print('XpYp shape is: ', XpYp.shape)

IM_T = np.concatenate((IM_T, XpYp), axis=2)
print('IM_T shape is: ', IM_T.shape)

In [ ]:
print(Xp[1,1])
print(Xp[1,2])
print(Xp[2,1])
print(Xp[2,2])

print(Yp[1,1])
print(Yp[1,2])
print(Yp[2,1])
print(Yp[2,2])

In [ ]:
# Read scene
RefScene = '/data/sushen/marinedebris/MARIDA/scenes/S2_20191201T030101_20191201T031728_T48MYU.tif'
ds_scene = gdal.Open(RefScene)
IM_scene = np.copy(ds_scene.ReadAsArray())
print('IM_scene shape: ', IM_scene.shape)

IM_scene = np.moveaxis(IM_scene, 0, -1)
print('IM_scene shape: ', IM_scene.shape)

In [ ]:
padfTransformScene = ds_scene.GetGeoTransform()
print('Transform is: ', padfTransformScene)

y_coords, x_coords = np.meshgrid(range(IM_scene.shape[0]), range(IM_scene.shape[1]), indexing='ij')

Xp = padfTransformScene[0] + x_coords*padfTransformScene[1] + y_coords*padfTransformScene[2]
Yp = padfTransformScene[3] + x_coords*padfTransformScene[4] + y_coords*padfTransformScene[5]

# shift to the center of the pixel
Xp -= padfTransformScene[5] / 2.0
Yp -= padfTransformScene[1] / 2.0
print('Xp shape is: ', Xp.shape)
print('Yp shape is: ', Yp.shape)

XpYp = np.dstack((Xp,Yp))
print('XpYp shape is: ', XpYp.shape)

IM_scene = np.concatenate((IM_scene, XpYp), axis=2)
print('IM_scene shape is: ', IM_scene.shape)

In [ ]:
print(Xp[1,1])
print(Xp[1,2])
print(Xp[2,1])
print(Xp[2,2])

print(Yp[1,1])
print(Yp[1,2])
print(Yp[2,1])
print(Yp[2,2])

In [ ]:
import geopandas as gpd
import rasterio as rio

In [ ]:
gdf = gpd.read_file('/data/sushen/marinedebris/MARIDA/shapefiles/S2_1-12-19_48MYU.shp')
gdf.head(15)

In [ ]:
with rio.open('/data/sushen/marinedebris/MARIDA/patches/S2_1-12-19_48MYU/S2_1-12-19_48MYU_0.tif') as src:
    crs = src.crs
    width = src.width
    height = src.height
    transform = src.transform
    profile = src.profile

    print(transform)